## Tratamento dos dados

### Importando bibliotecas

In [1]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import adjusted_mutual_info_score
import pandas as pd
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
def inicializacao_matriz_pertinencia(num_amostras, num_clusters):
    matriz_pertinencia = np.random.rand(num_amostras, num_clusters) # gera uma matriz inicial aleatória com valores entre 0 e 1
    matriz_pertinencia = matriz_pertinencia / matriz_pertinencia.sum(axis=1, keepdims=True) # normalização da matriz pra garantir que a soma dos graus dê um
    return matriz_pertinencia
def inicializacao_pesos(num_variaveis):
    pesos = np.ones(num_variaveis) / num_variaveis # pesos iguais somam 1
    return pesos
def atualizacao_centroides(dados, matriz_pertinencia, m):
    matriz_pertinencia_m = matriz_pertinencia ** m # preparação dos graus de pertinência
    centroides = np.dot(matriz_pertinencia_m.T, dados) / np.sum(matriz_pertinencia_m.T, axis=1, keepdims=True) # fórmula para o cálculo dos centroides
    return centroides
def atualizacao_matriz_pertinencia(dados, centroides, pesos, m, beta):
    diff_sq = (dados[:, np.newaxis, :] - centroides) ** 2 # diferença ao quadrado (x_kj - y_ij)^2
    pesos_potencia = pesos ** beta
    weighted_diff_sq = pesos_potencia * diff_sq # aplica o peso elevado
    matriz_distancias = np.sum(weighted_diff_sq, axis=2) # d_ik^(w)^2 = sum_j( w_j^beta * (x_kj - y_ij)^2 )
    matriz_distancias = np.fmax(matriz_distancias, np.finfo(np.float64).eps)
    potencia = 1.0 / (m - 1)
    matriz_distancias_inversa = 1 / matriz_distancias # fórmula transformada: u_ik = (1/d_ik^2)^P / sum_h( (1/d_hk^2)^P )
    num = matriz_distancias_inversa ** potencia
    den = np.sum(num, axis=1, keepdims=True) # sum_h( (1/d_hk^2)^P )
    nova_matriz_pertinencia = num / den
    nova_matriz_pertinencia = nova_matriz_pertinencia / np.sum(nova_matriz_pertinencia, axis=1, keepdims=True)
    return nova_matriz_pertinencia
def atualizacao_pesos(dados, centroides, matriz_pertinencia, beta):
    matriz_pertinencia_broadcast = matriz_pertinencia[:, :, np.newaxis] 
    diff_sq = (dados[:, np.newaxis, :] - centroides) ** 2 # diferença ao quadrado (x_kj - y_ij)^2
    weighted_diff_sq = matriz_pertinencia_broadcast * diff_sq # multiplicação
    D_j = np.sum(weighted_diff_sq, axis=(0, 1))
    D_j = np.fmax(D_j, np.finfo(np.float64).eps)
    beta_exponente = 1.0 / np.fmax(beta - 1, np.finfo(np.float64).eps) # 1 / (beta - 1)
    razao = D_j[:, np.newaxis] / D_j[np.newaxis, :] # D_j / D_t
    razao_potencia = razao ** beta_exponente
    soma_termos = np.sum(razao_potencia, axis=1) # sum_t
    pesos = 1.0 / soma_termos # tem no paper original
    pesos = pesos / np.sum(pesos) # pesos somam 1
    return pesos
def fcm(dados, num_clusters, m=2, beta=2, max_iter=10**6, erro=1e-9):
    num_amostras, num_variaveis = dados.shape
    matriz_pertinencia = inicializacao_matriz_pertinencia(num_amostras, num_clusters)
    pesos = inicializacao_pesos(num_variaveis)
    for _ in range(max_iter): # primeiro critério de parada
        centroides = atualizacao_centroides(dados, matriz_pertinencia, m)
        nova_matriz_pertinencia = atualizacao_matriz_pertinencia(dados, centroides, pesos, m, beta)
        pesos = atualizacao_pesos(dados, centroides, nova_matriz_pertinencia, beta)
        if np.linalg.norm(nova_matriz_pertinencia - matriz_pertinencia) < erro: # segundo critério de parada
            break
        matriz_pertinencia = nova_matriz_pertinencia
    return centroides, matriz_pertinencia, pesos
def indice_rand(labels, predicted_labels):
    return adjusted_rand_score(labels, predicted_labels)
def simulacao_monte_carlo(dados, labels, num_clusters, num_trials):
    ari = []
    ami = []
    for _ in range(num_trials):
        centroides, matriz_pertinencia, pesos = fcm(dados, num_clusters)
        predicted_labels = np.argmax(matriz_pertinencia, axis=1)
        idx_rand = indice_rand(labels, predicted_labels)
        ari.append(idx_rand)
        ami_rand = adjusted_mutual_info_score(labels, predicted_labels)
        ami.append(ami_rand)
    mean_ari = np.mean(ari)
    std_ari = np.std(ari)
    mean_ami = np.mean(ami)
    std_ami = np.std(ami)
    return mean_ari, std_ari, mean_ami, std_ami

In [3]:
def gerar_configuracao(mu_list, sigma_list, tamanhos, config_id):
    dfs = []
    for i, (mu, sigma2, n) in enumerate(zip(mu_list, sigma_list, tamanhos)):
        Sigma = np.diag(sigma2)
        data = np.random.multivariate_normal(mu, Sigma, n)
        df = pd.DataFrame(data, columns=["x1", "x2"])
        df["class"] = i + 1
        dfs.append(df)
    df_config = pd.concat(dfs, ignore_index=True)
    df_config["config"] = config_id
    return df_config

np.random.seed(42)  # reprodutibilidade

# Configuração 1
mu_1 = [[5, 0], [15, 5], [18, 14]]
sigma2_1 = [[81, 9], [9, 100], [25, 36]]
n1 = [200, 100, 50]
df1 = gerar_configuracao(mu_1, sigma2_1, n1, config_id=1)
l1 = "Classes elípticas de tamanhos diferentes"

# Configuração 2
mu_2 = [[0, 0], [30, 0], [12, 25]]
sigma2_2 = [[100, 100], [49, 49], [16, 16]]
n2 = [200, 100, 50]
df2 = gerar_configuracao(mu_2, sigma2_2, n2, config_id=2)
l2 = "Classes esféricas de tamanhos diferentes"

# Configuração 3
mu_3 = [[0, 0], [15, 5], [15, -5]]
sigma2_3 = [[100, 4], [100, 4], [100, 4]]
n3 = [100, 100, 100]
df3 = gerar_configuracao(mu_3, sigma2_3, n3, config_id=3)
l3 = "Classes elípticas de tamanhos iguais"

# Configuração 4
mu_4 = [[0, 0], [15, 0], [-15, 0]]
sigma2_4 = [[16, 16], [16, 16], [16, 16]]
n4 = [100, 100, 100]
df4 = gerar_configuracao(mu_4, sigma2_4, n4, config_id=4)
l4 = "Classes esféricas de tamanhos iguais"

# Configuração 5
mu_5 = [[5, 0], [15, 5], [10, -7], [3, 15]]
sigma2_5 = [[81, 9], [9, 100], [49, 16], [25, 25]]
n5 = [50, 50, 50, 50]
df5 = gerar_configuracao(mu_5, sigma2_5, n5, config_id=5)
l5 = "3 classes elípticas e 1 esférica"

# Configuração 6
mu_6 = [[5, 0], [15, 5], [12, -12], [7, 17]]
sigma2_6 = [[81, 9], [9, 100], [16, 16], [25, 25]]
n6 = [50, 50, 50, 50]
df6 = gerar_configuracao(mu_6, sigma2_6, n6, config_id=6)
l6 = "2 classes elípticas e 2 esféricas"

# Configuração 7
mu_7 = [[0, 0], [18, 0], [-18, 0], [0, -12]]
sigma2_7 = [[12, 12], [20, 20], [16, 16], [81, 20]]
n7 = [50, 50, 50, 50]
df7 = gerar_configuracao(mu_7, sigma2_7, n7, config_id=7)
l7 = "1 classe elíptica e 3 esféricas"

In [4]:
i = 1
for df in [df1, df2, df3, df4, df5, df6, df7]:
    np.random.seed(42)  # reprodutibilidade
    df.drop("config", axis=1, inplace=True)  # remove a coluna de configuração
    labels = df["class"].values
    num_clusters = len(df[df['class'] != 0]['class'].unique())  # número de clusters, excluindo o ruído
    df.drop("class", axis=1, inplace=True)  # remove a coluna de classe
    dados = df.to_numpy()
    num_trials = 100
    mean_ari, std_ari, mean_ami, std_ami = simulacao_monte_carlo(dados, labels, num_clusters, num_trials)
    print(f"Configuração {i}")
    print(f"ARI mean: {mean_ari:.4f}")
    print(f"ARI std: {std_ari:.4f}")
    print(f"AMI mean: {mean_ami:.4f}")
    print(f"AMI std: {std_ami:.4f}")
    print("")
    i +=1

Configuração 1
ARI mean: 0.2978
ARI std: 0.0000
AMI mean: 0.4096
AMI std: 0.0000

Configuração 2
ARI mean: 0.4874
ARI std: 0.1212
AMI mean: 0.5173
AMI std: 0.0968

Configuração 3
ARI mean: 0.6417
ARI std: 0.0000
AMI mean: 0.6044
AMI std: 0.0000

Configuração 4
ARI mean: -0.0050
ARI std: 0.0000
AMI mean: -0.0049
AMI std: 0.0000

Configuração 5
ARI mean: 0.1320
ARI std: 0.0000
AMI mean: 0.2081
AMI std: 0.0000

Configuração 6
ARI mean: 0.1568
ARI std: 0.0000
AMI mean: 0.2235
AMI std: 0.0000

Configuração 7
ARI mean: 0.2324
ARI std: 0.0000
AMI mean: 0.2996
AMI std: 0.0000

